In [1]:
%pip install numba

     |████████████████████████████████| 3.4 MB 4.6 MB/s eta 0:00:01
     |████████████████████████████████| 34.5 MB 5.6 MB/s eta 0:00:012
     |████████████████████████████████| 16.8 MB 8.8 MB/s eta 0:00:01
You should consider upgrading via the '/home/mh/.pyenv/versions/3.10.0rc2/envs/nyu/bin/python3.10 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import numba

In [55]:
@numba.njit
def compute_greedy(N, max_change, denos):
    num = 0
    den = 0
    for change in range(max_change):
        weight = N if change % 5 == 0 else 1
        for val in denos[::-1]:
            k = change // val
            change %= val
            num += weight * k
        den += weight
    return num / den


@numba.njit
def score(req_coins, N):
    req_coins_5k = req_coins[::5]
    num = sum(req_coins) + (N-1) * sum(req_coins_5k)
    den = len(req_coins) + (N-1) * len(req_coins_5k)
    return num / den


@numba.njit
def compute_dp(N, max_change, denos):
    ans = [100000]*240
    ans[0] = 0
    for change in range(max_change):
        for coin in denos:
            if coin <= change:
                ans[change] = min(ans[change], ans[change-coin]+1)
    return score(ans, N)


@numba.njit
def solve(N, max_change, compute):
    best_score = 100000000
    best_denos = None  # (0,0,0,0) 
    c1 = 1
    for c2 in range(c1+1, max_change):
        for c3 in range(c2+1, max_change):
            for c4 in range(c3+1, max_change):
                denos = (c1,c2,c3,c4)
                new_score = compute(N, max_change, denos)
                if best_score > new_score:
                    best_score = new_score
                    best_denos = denos
    print(best_score, best_denos)

In [56]:
%%time
solve(5, 240, compute_greedy)

4.861111111111111 (1, 5, 20, 70)
CPU times: user 10.7 s, sys: 27.7 ms, total: 10.7 s
Wall time: 10.7 s


In [57]:
%%time
solve(5, 240, compute_dp)

4.777777777777778 (1, 5, 35, 55)
CPU times: user 5.32 s, sys: 7.88 ms, total: 5.32 s
Wall time: 5.31 s
